### Next steps:

TODO:
1. Develop method for measuring wave propogation speed in image coordinates,  then find the flattened homography that makes this as constant as possible.
1. Create a pipeline that generates animations for each stage of the training process (normalization, clipping, learning), and compiles into a demo video.
1. Get tensorboard metrics logging working with this logger class: https://github.com/PyTorchLightning/pytorch-lightning/issues/1228#issuecomment-622963564


### Now get optuna optimization trials working

TODO:
1. Improve early stopping such that I have control over patience parameter and ensure the _minimum_ validation loss is passed to the optuna study.

The most well-regularized hyperparameters for simple (`mse_loss` only) ended up being 
256 hidden features, 3.7995 first_omega_0, 2.9312 hidden_omega_0

With wavefunc loss, `squared_slowness` of around 0.5 may be close

```
Finished trial#26 with value: 0.09781524538993835 with parameters: 
{'first_omega_0': 4.839289222946841, 'hidden_omega_0': 13.756932872278343, 'squared_slowness': 0.27488941275825124, 'wave_loss_scale': 9.252313787089657e-08}
```

In [ ]:
import os
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet
from optuna.integration import PyTorchLightningPruningCallback
from surfbreak.studies import run_waveform_hyperparam_search, MetricsCallback
LOGDIR = '../tmp/testlogs'
MODELDIR = os.path.join(LOGDIR, 'opt_models')

def objective(trial):
    checkpoint_callback = pl.callbacks.ModelCheckpoint( # Filenames for each trial must be made unique
        os.path.join(MODELDIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss")
    tb_logger = pl.loggers.TensorBoardLogger(LOGDIR+'/', name="optuna")
    metrics_callback = MetricsCallback()     # Simple callback that saves metrics from each validation step.
    
    pl.seed_everything(42)
    trainer = pl.Trainer(logger=tb_logger, limit_val_batches=3,
                         max_epochs=2, 
                         gpus=1 if torch.cuda.is_available() else None,
                         callbacks=[metrics_callback],
                         early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss"),
                        )

    wavefunc_model = LitSirenNet(hidden_features=trial.suggest_categorical('hidden_features', [128, 256, 380]), #256,
                                 hidden_layers=trial.suggest_categorical('hidden_layers', [3]), #3,
                                 first_omega_0=trial.suggest_uniform('first_omega_0', 0.5, 2.5), #1.5, 
                                 hidden_omega_0=trial.suggest_uniform('hidden_omega_0', 8.5, 15), #11.7,
                                 squared_slowness=trial.suggest_uniform('squared_slowness', 0.20, 0.70), #0.23,
                                 steps_per_vid_chunk=10, 
                                 learning_rate=1e-4, grad_loss_scale=0, 
                                 wavefunc_loss_scale=trial.suggest_loguniform('wavefunc_loss_scale', 5e-9, 1e-7), #2e-8,
                                 wavespeed_loss_scale=trial.suggest_loguniform('wavespeed_loss_scale', 1e-4, 1e-2), #1e-3,
                                 xrange=(0,200), timerange=(0,3*10), chunk_duration=10, chunk_stride=10)
                                 # With default settings wavezone image dimensions are (y=139, x=1515)

    trainer.fit(wavefunc_model)
    return metrics_callback.metrics[-1]["val_loss"].item()


study = run_waveform_hyperparam_search(objective, n_trials=3, timeout=2*60, model_dir=MODELDIR, 
                                       prune=False, n_startup_trials=2, n_warmup_steps=5)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 13:13:26,704] Finished trial#0 with value: 0.19167518615722656 with parameters: {'hidden_features': 256, 'hidden_layers': 3, 'first_omega_0': 1.248046093392649, 'hidden_omega_0': 14.838968497149384, 'squared_slowness': 0.381381985610618, 'wavefunc_loss_scale': 3.1740983635383545e-08, 'wavespeed_loss_scale': 0.0002281624447899972}. Best is trial#0 with value: 0.19167518615722656.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 436 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 13:13:43,577] Finished trial#1 with value: 0.20219139754772186 with parameters: {'hidden_features': 380, 'hidden_layers': 3, 'first_omega_0': 1.1556850000715602, 'hidden_omega_0': 11.330566013865619, 'squared_slowness': 0.3681701069183505, 'wavefunc_loss_scale': 5.636494641482703e-08, 'wavespeed_loss_scale': 0.002031922228778422}. Best is trial#0 with value: 0.19167518615722656.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 50 K   | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


[I 2020-07-05 13:14:00,394] Finished trial#2 with value: 0.18954741954803467 with parameters: {'hidden_features': 128, 'hidden_layers': 3, 'first_omega_0': 1.1841127152543858, 'hidden_omega_0': 11.358083756313386, 'squared_slowness': 0.38535743636398334, 'wavefunc_loss_scale': 4.548020166320603e-08, 'wavespeed_loss_scale': 0.0004287541327616621}. Best is trial#2 with value: 0.18954741954803467.


Number of finished trials: 3
Best trial was #2:
  Value: 0.18954741954803467
  Params: 
    hidden_features: 128
    hidden_layers: 3
    first_omega_0: 1.1841127152543858
    hidden_omega_0: 11.358083756313386
    squared_slowness: 0.38535743636398334
    wavefunc_loss_scale: 4.548020166320603e-08
    wavespeed_loss_scale: 0.0004287541327616621


In [ ]:
study.best_params

{'hidden_features': 128,
 'hidden_layers': 3,
 'first_omega_0': 1.1841127152543858,
 'hidden_omega_0': 11.358083756313386,
 'squared_slowness': 0.38535743636398334,
 'wavefunc_loss_scale': 4.548020166320603e-08,
 'wavespeed_loss_scale': 0.0004287541327616621}

In [ ]:
sdf = study.trials_dataframe()
top_trials = sdf.sort_values(by='value')[:5]
top_trials

,number,value,datetime_start,datetime_complete,duration,params_first_omega_0,params_hidden_features,params_hidden_layers,params_hidden_omega_0,params_squared_slowness,params_wavefunc_loss_scale,params_wavespeed_loss_scale,state
2,2,0.189547,2020-07-05 13:13:43.580345,2020-07-05 13:14:00.394124,00:00:16.813779,1.184113,128,3,11.358084,0.385357,4.548020e-08,0.000429,COMPLETE
0,0,0.191675,2020-07-05 13:13:07.087647,2020-07-05 13:13:26.704373,00:00:19.616726,1.248046,256,3,14.838968,0.381382,3.174098e-08,0.000228,COMPLETE
1,1,0.202191,2020-07-05 13:13:26.706818,2020-07-05 13:13:43.576777,00:00:16.869959,1.155685,380,3,11.330566,0.368170,5.636495e-08,0.002032,COMPLETE


In [ ]:
print(top_trials.mean())
print(top_trials.std())

number                                              1
value                                        0.194471
duration                       0 days 00:00:17.766821
params_first_omega_0                          1.19595
params_hidden_features                        254.667
params_hidden_layers                                3
params_hidden_omega_0                         12.5092
params_squared_slowness                      0.378303
params_wavefunc_loss_scale                4.45287e-08
params_wavespeed_loss_scale                0.00089628
dtype: object
number                                              1
value                                      0.00676989
duration                       0 days 00:00:01.602310
params_first_omega_0                        0.0473043
params_hidden_features                        126.005
params_hidden_layers                                0
params_hidden_omega_0                         2.01768
params_squared_slowness                     0.0089978
params_wavefun